# Settings

In [ ]:
NB_TESTS_PER_TARGET = 100
MAX_VARIABLE_FOR_BRUTEFORCE = 15

COMPUTE_COMPLEMENTARY = True
DEBUG = False

In [ ]:
from pylfit.postprocessing import compute_counterfactuals, bruteforce_counterfactuals
from pylfit.preprocessing.boolean_network import dmvlp_from_boolean_network_file
from pylfit.preprocessing import discrete_state_transitions_dataset_from_array

import random
import time
import pandas
import seaborn as sns
from matplotlib import colors as mcolors

import matplotlib.pyplot as plt

# Compute counterfactual with bruteforce and proposal

In [ ]:
benchmarks = [
    "../../benchmarks/boolean_networks/pyboolnet/bio/faure_cellcycle.bnet", 
    "../../benchmarks/boolean_networks/pyboolnet/bio/dinwoodie_life.bnet",
    "../../benchmarks/boolean_networks/pyboolnet/bio/irons_yeast.bnet",
    "../../benchmarks/boolean_networks/pyboolnet/bio/tournier_apoptosis.bnet",
    "../../benchmarks/boolean_networks/pyboolnet/bio/saadatpour_guardcell.bnet",
    "../../benchmarks/boolean_networks/pyboolnet/bio/dahlhaus_neuroplastoma.bnet",
    #"../../benchmarks/boolean_networks/boolenet/budding_yeast.net",
    "../../benchmarks/boolean_networks/boolenet/arabidopsis.net",
    "../../benchmarks/boolean_networks/boolenet/fission_yeast.net",
    "../../benchmarks/boolean_networks/boolenet/thelper.net",
    "../../benchmarks/boolean_networks/boolenet/tcr.net"
    ]

data = []

for bn_file_path in benchmarks:
    print("Loading DMVLP from Boolean network file:", bn_file_path)
    model = dmvlp_from_boolean_network_file(bn_file_path, compute_complementary_rules=COMPUTE_COMPLEMENTARY)
    #model.summary()
    print("variables:", len(model.features))

    # Make the program complete by learning from its transitions
    if not COMPUTE_COMPLEMENTARY:
        print("All states transitions (synchronous): ")
        feature_states = model.feature_states()
        transitions = []
        prediction = model.predict(feature_states, semantics="synchronous", default=[(var,["0"]) for var in model.features])
        for s1 in prediction:
            for s2 in prediction[s1]:
                transitions.append( (s1, s2) )

        print(len(transitions))

        dataset = discrete_state_transitions_dataset_from_array(
        data=transitions,
        feature_names=[var for var,vals in model.features],
        target_names=[var for var,vals in model.targets])

        model.fit(dataset)
        #model.summary()
    
    #model.summary()

    for target_id in range(len(model.targets)):
        
        for test in range(NB_TESTS_PER_TARGET):
            target_name = model.targets[target_id][0]
            s = tuple([random.choice(vals) for var, vals in model.features])
            target_value = [s2 for s2 in model.predict([s])[s]][0][target_id]
            desired_value = "0"
            if target_value == "0":
                desired_value == "1"

            #print(s, target_name, target_value, desired_value)

            if DEBUG:
                solutions = []
                data.append(["Naïve enumeration", bn_file_path, len(model.features), s, target_id, target_value, desired_value, solutions, len(solutions), 2])
                data.append(["CELOS", bn_file_path, len(model.features), s, target_id, target_value, desired_value, solutions, len(solutions), 1])

            else:
                if len(model._features) <= MAX_VARIABLE_FOR_BRUTEFORCE:
                    begin = time.time()
                    solutions = bruteforce_counterfactuals(model, s, target_name, [target_value], [desired_value])[desired_value]
                    run_time = time.time() - begin
                    data.append(["Naïve enumeration", bn_file_path, len(model.features), s, target_id, target_value, desired_value, solutions, len(solutions), run_time])

                begin = time.time()
                solutions = compute_counterfactuals(model, s, target_name, [target_value], [desired_value])[desired_value]
                run_time = time.time() - begin
                data.append(["CELOS", bn_file_path, len(model.features), s, target_id, target_value, desired_value, solutions, len(solutions), run_time])

            #print(solutions)
            #print("\r"+str(test+1)+"/"+str(NB_TESTS_PER_TARGET))
        
df = pandas.DataFrame(data, columns=["method", "file", "variables", "feature_state", "target_id",  "target_value", "desired_value", "solutions", "nb_solutions", "run_time"])
display(df)

In [ ]:
df[df["nb_solutions"]>0]

# Visualize performance

In [ ]:
BRUTE_FORCE_COLOR = mcolors.to_rgba("#C6110B") #"lightcoral"
GULA_COLOR = mcolors.to_rgba("#0B7FF3") #"skyblue"
WHIS_VALUE = 10000000

In [ ]:
#df_brute_force = df_semantics[df_semantics["method"] == "brute_force"]
ax = sns.boxplot(x="variables", y="run_time", hue="method", data=df,
                #whiskerprops = dict(color=BRUTE_FORCE_COLOR, linewidth=2.0, alpha= 0.8),
                #capprops = dict(color=BRUTE_FORCE_COLOR, linewidth=2.0, alpha= 0.8),
                whis=WHIS_VALUE,
                palette="Blues",
                log_scale=False)

In [ ]:
df_tmp = df.copy()
df_tmp["run_time"] += 0.001
ax = sns.boxplot(x="variables", y="run_time", hue="method", data=df_tmp,
                #whiskerprops = dict(color=BRUTE_FORCE_COLOR, linewidth=2.0, alpha= 0.8),
                #capprops = dict(color=BRUTE_FORCE_COLOR, linewidth=2.0, alpha= 0.8),
                whis=WHIS_VALUE,
                palette="Blues",
                log_scale=True)

plt.savefig("bn_benchmarks_run_times.pdf", bbox_inches='tight')